# 파이썬에서 한글 처리 

All rights reserved, 2021, By **Youn-Sik Hong**. 수업 목적으로만 활용 가능.

- 파이썬에서는 utf-8 코드 체계를 사용    
    - utf-8 : unicode를 완벽하게 지원하는 코드 체계.
    - Unicode 문자코드 차트 : http://www.unicode.org/charts/
        - 한글은 East Asian Scripts에서 Hangul Jamo에서 찾으면 됨.
- 파일을 읽어오거나 처리 결과를 파일로 저장할 때 해당 문자열의 인코딩 방식에 맞는 변환이 필요.
    - decode : latin-2, utf-8 등으로 인코딩한 파일 --> 파이썬 내부로 가져올 때 unicode로 변환하는 과정.
    - encode : 파이썬 내부에서 처리한 unicode 문자열 --> utf-8, cp949 등으로 인코딩해서 파일로 저장하는 과정.

## 한글 코드 변환 

In [1]:
import sys

print(sys.getdefaultencoding())

utf-8


- cp949는 완성형 코드이며, utf-8(unicode)은 조합형 코드를 사용. 
- 한글 조합코드: 초성코드+중성코드+종성코드의 조합으로 한글 음절(syllable)을 구성하는 것을 말함.
- 아래 출력에서 b는 binary, x는 hexadecimal(16진수)을 의미.

In [5]:
print('홍'.encode('cp949'))
print(b'\xc8\xab'.decode('cp949'))

b'\xc8\xab'
홍


In [8]:
print("홍".encode("utf-8"))
print(b'\xed\x99\x8d'.decode('utf-8'))

b'\xed\x99\x8d'
홍


- **김 영랑** 시인의 시 '오매, 단풍들것네'
    - *오매 단풍들것네 장광에 골불은 감닙 날러오아 누이는 놀란 듯이 치어다보며 오매 단풍들것네.*

In [9]:
hangul_sent = '오매 단풍들것네' 
print(hangul_sent)

오매 단풍들것네


시스템 내부 또는 파일에 저장될 때 어떻게 표현될까? - \\uxxxx (x는 16진수)

encode 메소드에서 **unicode_escape** 옵션을 설정하면 음절 단위로 저장. '오','매',' ','단','풍','들','것','네'

In [10]:
print(hangul_sent.encode('unicode_escape'))

b'\\uc624\\ub9e4 \\ub2e8\\ud48d\\ub4e4\\uac83\\ub124'


In [13]:
print('\uc624')
print('\uc124')

오
설


- encode 메소드에서 **utf-8** 옵션을 설정하면, 한글 음절을 구성하는 **자모** (자음모음) 단위로 저장. 
- '오'는 'ㅇ'+'ㅗ'+'받침 없음'과 같이 3개 코드로 변환되어 저장.
    - 모든 한글 음절은 각각 3개의 자모 코드로 변환. 7개 한글 음절이기 때문에 21개 코드로 바뀜.
    - \\x는 16진수를 표시하는 prefix이기 때문에 다음 2자리 16진수 수가 자모 코드.
- space 문자는 화면에 보이는 그대로 저장됨. space 문자는 ASCII(7bits)의 128개 코드에 포함되기 때문.

In [14]:
print(hangul_sent.encode('utf-8')) 
#print(hangul_sent.encode()) #default는 utf-8로 인코딩

b'\xec\x98\xa4\xeb\xa7\xa4 \xeb\x8b\xa8\xed\x92\x8d\xeb\x93\xa4\xea\xb2\x83\xeb\x84\xa4'


In [17]:
print(b'\xec\x98\xa4'.decode('utf-8'))

오


In [18]:
import unicodedata

아래 예에서 by, Young, Rang, Kim과 space 및 dash (-)문자는 ASCII 코드에 포함되기 때문에 출력되지 않음.

In [19]:
hangul_sent2 = '오매 단풍들것네 by Young-Rang Kim'
for c in hangul_sent2: 
    if ord(c) > 127:  #ASCII 코드가 아닌 경우 unicode로 변환
        print('{} U+{:04x} {}'.format(c.encode('utf8'), ord(c), unicodedata.name(c)))

b'\xec\x98\xa4' U+c624 HANGUL SYLLABLE O
b'\xeb\xa7\xa4' U+b9e4 HANGUL SYLLABLE MAE
b'\xeb\x8b\xa8' U+b2e8 HANGUL SYLLABLE DAN
b'\xed\x92\x8d' U+d48d HANGUL SYLLABLE PUNG
b'\xeb\x93\xa4' U+b4e4 HANGUL SYLLABLE DEUL
b'\xea\xb2\x83' U+ac83 HANGUL SYLLABLE GEOS
b'\xeb\x84\xa4' U+b124 HANGUL SYLLABLE NE


초성코드+중성코드+종성코드로 만들어진 한글 음절과 unicode로 정의한 한글 음절의 디코딩 결과를 확인해 보자.

In [20]:
print(b'\xec\x98\xa4'.decode('utf-8'), '\uc624')

오 오


## 한글 호환 자모(Hangul Compatibility Jamo)
- 초성이나 받침이 아닌 자모(자음, 모음)만을 위한 코드이며, 자모가 정상 크기로 출력됨.

In [21]:
def show_hangulcode(s, e, f=False):
    if f:
        print(s, e, e-s+1, hex(s), hex(e))
    print() #한글이 차지하는 공간이 넓어서 여유 공간(padding)을 확보
    for i in range(s, e):
        print(chr(i), end=' ')    

In [22]:
start = int('3131', 16) #16진수를 10진수로 변환
end = int('314e', 16)
show_hangulcode(start, end)


ㄱ ㄲ ㄳ ㄴ ㄵ ㄶ ㄷ ㄸ ㄹ ㄺ ㄻ ㄼ ㄽ ㄾ ㄿ ㅀ ㅁ ㅂ ㅃ ㅄ ㅅ ㅆ ㅇ ㅈ ㅉ ㅊ ㅋ ㅌ ㅍ 

In [24]:
start = int('314f', 16) 
end = int('3163', 16)
show_hangulcode(start, end,f=True)

12623 12643 21 0x314f 0x3163

ㅏ ㅐ ㅑ ㅒ ㅓ ㅔ ㅕ ㅖ ㅗ ㅘ ㅙ ㅚ ㅛ ㅜ ㅝ ㅞ ㅟ ㅠ ㅡ ㅢ 

## unicode에서 한글 음절
- unicode에서 한글 음절은 모두 19(초성)x21(중성)x28(종성) = 11,172자이며, 첫 음절은 '가', 마지막 음절은 '힣'.
- 첫 음절 '가'의 코드는 44032(0xAC00), 마지막 음절 '힣'의 코드는 55203(0xD7A3).
- 초성 시작 위치: 0x1100(4352)
- 중성 시작 위치: 0x1161(4449)
- 종성 시작 위치: 0x11A7(4519)

In [25]:
a,b,c = int('1100', 16), int('1161', 16), int('11A7', 16)
a, b, c

(4352, 4449, 4519)

unicode에서 한글 초성: 19자 = 홑자음 14자 + 쌍자음 5자

In [26]:
start = int('1100', 16) 
end = start+19
print()
show_hangulcode(start, end)



ᄀ ᄁ ᄂ ᄃ ᄄ ᄅ ᄆ ᄇ ᄈ ᄉ ᄊ ᄋ ᄌ ᄍ ᄎ ᄏ ᄐ ᄑ ᄒ 

unicode에서 한글 중성: 21자 = 홑모음 10자 + 겹모음 11자

In [27]:
start = int('1161', 16) 
end = start+21
print()
show_hangulcode(start, end)



ᅡ ᅢ ᅣ ᅤ ᅥ ᅦ ᅧ ᅨ ᅩ ᅪ ᅫ ᅬ ᅭ ᅮ ᅯ ᅰ ᅱ ᅲ ᅳ ᅴ ᅵ 

unicode에서 한글 종성: 28자 = 받침없음(1자) + 자음 14자 + 쌍자음 2자 + 겹자음 11자

In [28]:
start = int('11a7', 16) 
end = start+28
print()
show_hangulcode(start+1, end) #공백 문자를 출력하지 않음.



ᆨ ᆩ ᆪ ᆫ ᆬ ᆭ ᆮ ᆯ ᆰ ᆱ ᆲ ᆳ ᆴ ᆵ ᆶ ᆷ ᆸ ᆹ ᆺ ᆻ ᆼ ᆽ ᆾ ᆿ ᇀ ᇁ ᇂ 

In [29]:
print(ord('가'), ord('힣')) #ord: 한글 음절에 할당된 unicode를 정수로 출력

44032 55203


In [30]:
print(chr(44032), hex(44032)) #hex: 10진수를 16진수로 변환
print(chr(55203), hex(55203)) #chr: 정수 코드를 문자로 변환

가 0xac00
힣 0xd7a3


In [31]:
han_syllable_1 = '\uac00' #unicode를 문자열(str)로 변환
han_syllable_11172 = '\ud7a3'
print(type(han_syllable_1))
print(han_syllable_1, han_syllable_11172)

<class 'str'>
가 힣


In [32]:
print(han_syllable_1.encode('utf8'), '가'.encode('utf8'))
print(han_syllable_11172.encode('utf8'), '힣'.encode('utf8'))

b'\xea\xb0\x80' b'\xea\xb0\x80'
b'\xed\x9e\xa3' b'\xed\x9e\xa3'


## 한글 자모를 결합하여  음절 완성

In [33]:
# 초성: 19자
uni_choSung = ['ㄱ', 'ㄲ', 'ㄴ', 'ㄷ', 'ㄸ', 'ㄹ', 'ㅁ', 'ㅂ', 'ㅃ', 
               'ㅅ', 'ㅆ', 'ㅇ', 'ㅈ', 'ㅉ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ']
# 중성: 21자
uni_joongSung = ['ㅏ', 'ㅐ', 'ㅑ', 'ㅒ', 'ㅓ', 'ㅔ', 'ㅕ', 'ㅖ', 'ㅗ', 
                 'ㅘ', 'ㅙ', 'ㅚ', 'ㅛ', 'ㅜ', 'ㅝ', 'ㅞ', 'ㅟ', 'ㅠ', 'ㅡ', 'ㅢ', 'ㅣ']
# 종성: 28자(맨 앞의 공백 문자는 받침 없음)
uni_jongSung = [' ', 'ㄱ', 'ㄲ', 'ㄳ', 'ㄴ', 'ㄵ', 'ㄶ', 'ㄷ', 'ㄹ', 
                 'ㄺ', 'ㄻ', 'ㄼ', 'ㄽ', 'ㄾ', 'ㄿ', 'ㅀ', 'ㅁ', 'ㅂ', 
                 'ㅄ', 'ㅅ', 'ㅆ', 'ㅇ', 'ㅈ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ']

In [34]:
def get_index(ch_set, ch):
    lim = len(ch_set)
    for i in range(lim):
        if ch == ch_set[i]:
            return i
    return -1 #not found

In [1]:
def cal_uniCode10 (cho, joong, jong):
    i = j = k = 0
    i = get_index(uni_choSung, cho) #'초성'(cho)이 choSung 테이블에서 몇 번째 위치에 해당하는가?
    j = get_index(uni_joongSung, joong) 
    k = get_index(uni_jongSung, jong) 
    return i*21*28 + j*28 + k + 44032

In [2]:
full_name = []

#이름 첫 글자를 초,중,종성으로 분리 후 유니코드를 구함 
#예: 홍 -> "ㅎ','ㅗ','ㅇ'
n_first =  cal_uniCode10('ㅎ', 'ㅗ', 'ㅇ')
full_name.append(chr(n_first))

#이름 중간 글자를 초,중,종성으로 분리 후 유니코드를 구함
n_middle = cal_uniCode10('ㅇ', 'ㅠ', 'ㄴ')
full_name.append(chr(n_middle))

#이름 끝 글자를 초,중,종성으로 분리 후 유니코드를 구함  
n_last = cal_uniCode10('ㅅ', 'ㅣ', 'ㄱ')
full_name.append(chr(n_last))

print(full_name)

NameError: name 'get_index' is not defined

## 한글 음절에서 자모 분리

In [3]:
def separateJamo (name):
    result = []
    
    for syllable in name:
        code = ord(syllable)
        if code >= 44032 and code <= 55203:          
        #if syllable >= '가' and syllable <= '힣':        
        #if re.match('[ㄱ-ㅎㅏ-ㅣ가-힣]', syllable) is not None:
            #code = ord(syllable) - 44032
            code -= 44032            
            jong = int(code % 28)
            
            code = int(code / 28)
            joong = int(code % 21)
            
            cho = int(code / 21)
            result.append(uni_choSung[cho])
            result.append(uni_joongSung[joong]) 
            result.append(uni_jongSung[jong])            
        else:
            result.append(syllable)
    print(result)

In [4]:
separateJamo('홍윤식')

NameError: name 'uni_choSung' is not defined